## Matched and mismatched experiments

In [ ]:
# import libraries
# ! pip install openpyxl
# ! pip install -U kaleido
# ! pip install ipykernel
# ! 
import pandas as pd
import numpy as np
import random as rd
#import subprocess
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from sklearn.metrics import jaccard_score
#import scipy as sp
#import scipy.stats as stats
import openpyxl
import os
import seaborn as sns

# avoid warnings
pd.options.mode.chained_assignment = None

In [ ]:
# set the paths
result_dir = '../server/results/matching/'
output_dir = '../server/outputs/matching/'
plot_dir = '../server/plots/matching/'
pos_dir = '../server/positions/matching/'

In [ ]:
# set variables
chromosomes_int = list(range(1,23))
chromosomes_str = [str(chrom) for chrom in chromosomes_int]
bk_list = ['1']#, '2', '3', '4', '5'] 
sc_list = ['129', '130', '131', '132', '133']

wrong_values = ['./.','.', '1', '0']
valid_values = ['1/1', '0/0', '0/1', '2/2', '1/2']

In [ ]:
# load datasets
bk1 = pd.read_csv('data/GM12878_gDNA_1.vcf', header=30,sep='\t', dtype='object')
bk2 = pd.read_csv('data/GM12878_gDNA_2.vcf', header=30,sep='\t', dtype='object')
bk3 = pd.read_csv('data/GM12878_gDNA_3.vcf', header=30,sep='\t', dtype='object')
bk4 = pd.read_csv('data/GM12878_gDNA_4.vcf', header=30,sep='\t', dtype='object')
bk5 = pd.read_csv('data/GM12878_gDNA_5.vcf', header=30,sep='\t', dtype='object')

sc129 = pd.read_csv('data/GM12878_SC_129.vcf.gz', header=30,sep='\t', dtype='object')
sc130 = pd.read_csv('data/GM12878_SC_130.vcf.gz', header=30,sep='\t', dtype='object')
sc131 = pd.read_csv('data/GM12878_SC_131.vcf.gz', header=30,sep='\t', dtype='object')
sc132 = pd.read_csv('data/GM12878_SC_132.vcf.gz', header=30,sep='\t', dtype='object')
sc133 = pd.read_csv('data/GM12878_SC_133.vcf.gz', header=30,sep='\t', dtype='object')

In [ ]:
for bk in bk_list:
    df_bk = pd.read_csv(f'data/GM12878_gDNA_{bk}.vcf', header=30,sep='\t', dtype='object')
    df_bk.iloc[:,9] = df_bk.iloc[:,9].apply(lambda x: x.split(':')[0])
    df_bk.dropna(inplace=True)
    df_bk = df_bk[~df_bk.iloc[:,9].isin(wrong_values)]
    df_bk.to_csv(os.path.join(output_dir, f'bulk_calls/GM12878_gDNA_calls_{bk}.vcf'), sep='\t', index=False)

In [ ]:
pd.read_csv(os.path.join(output_dir, 'bulk_calls/GM12878_gDNA_calls_2.vcf'),header=0,sep='\t', dtype='object').head(3)

1. considero solo le SNPs con valori di genotipo accettabili
2. per ogni combinazione tra bulk e single cell creo dataset che fanno match e mismatch
3. separo questi dataset per ciascun cromosoma considerando che beagle considera solo reference panels per ciascun cromosoma 

In [ ]:
for bk in bk_list:
    for sc in sc_list:
        
        print(f'starting merging datasets bulk {bk} and single cell {sc}')
        df_bk = pd.read_csv(f'data/GM12878_gDNA_{bk}.vcf', header=30,sep='\t', dtype='object')
        df_sing = pd.read_csv(f'data/GM12878_SC_{sc}.vcf.gz', header=30,sep='\t', dtype='object')

        column_names = df_bk.columns.to_list()
        
        df_bk = df_bk[df_bk.iloc[:,0].isin(chromosomes_str)]
        df_sing = df_sing[df_sing.iloc[:,0].isin(chromosomes_str)]

        merged_df = pd.concat([df_bk, df_sing.iloc[:,9]], axis=1)
        merged_df.iloc[:,9] = merged_df.iloc[:,9].apply(lambda x: x.split(':')[0])
        merged_df.iloc[:,10] = merged_df.iloc[:,10].apply(lambda x: x.split(':')[0])

        merged_df['FORMAT'] = 'GT'

        print(f'dimension before filtering: {merged_df.shape[0]}')

        merged_df['match'] = (merged_df.iloc[:,9] == merged_df.iloc[:,10]).astype(int)

        valid_values = ['1/1', '0/0', '0/1', '2/2', '1/2']
        merged_df_filt = merged_df[merged_df.iloc[:,9].isin(valid_values)]
        

        print(merged_df_filt.shape[0])

        merged_df_filt_matched = merged_df_filt[merged_df_filt['match']==1]
        merged_df_filt_mismatc = merged_df_filt[merged_df_filt['match']==0]

        merged_df_filt_matched.to_csv(os.path.join(output_dir, f'matched/GM12878_matched_bk{bk}_sc{sc}_chrs.vcf'), sep='\t', index=False)
        merged_df_filt_mismatc.to_csv(os.path.join(output_dir, f'mismatc/GM12878_mismatc_bk{bk}_sc{sc}_chrs.vcf.gz'), sep='\t', index=False)

        print('merging datasets finished')

        for chrom in chromosomes_str:
            # splitting chromosomes
            
            merged_df_filt_matched_chr = merged_df_filt_matched[merged_df_filt_matched['#CHROM'] == chrom]
            merged_df_filt_mismatc_chr = merged_df_filt_mismatc[merged_df_filt_mismatc['#CHROM'] == chrom]

            merged_df_filt_matched_chr.to_csv(os.path.join(output_dir, f'matched/GM12878_matched_bk{bk}_sc{sc}_chrom{chrom}.vcf'), sep='\t', index=False)
            merged_df_filt_mismatc_chr.to_csv(os.path.join(output_dir, f'mismatc/GM12878_mismatc_bk{bk}_sc{sc}_chrom{chrom}.vcf.gz'), sep='\t', index=False)

In [ ]:
df1 = pd.read_csv(os.path.join(output_dir, f'matched/GM12878_matched_bk{bk}_sc{sc}_chrom{chrom}.vcf'), header=0, sep='\t', dtype='object')
df2 = pd.read_csv(os.path.join(output_dir, f'mismatc/GM12878_mismatc_bk{bk}_sc{sc}_chrom{chrom}.vcf.gz'), header=0, sep='\t', dtype='object')

In [ ]:
df1.head(3)

In [ ]:
df2.head(3)

### Matched analysis

In [ ]:
perc = 10
table = []

for bk in bk_list:
    for sc in sc_list:
        for chrom in chromosomes_str:

            df = pd.read_csv(os.path.join(output_dir, f'matched/GM12878_matched_bk{bk}_sc{sc}_chrom{chrom}.vcf'), header=0,sep='\t', dtype='object')
            
            # seleziono le posizioni da mascherare con un seed e una percentuale fissata a 10/100
            rd.seed(1)
            pos = df['POS'].tolist()
            pos_seed = rd.sample(pos, round((len(pos)*(int(perc)/100))))

            # posizioni che maschero
            # position_masked sono le stesso del check
            position_masked = os.path.join(pos_dir, f'pos_masked_matched_{bk}_{sc}_chr{chrom}.txt')
            with open(position_masked,'w') as output:
                output.write(str(pos_seed))

            # creo i dataframe che mi servono da testare e check
            df_test = df[~df['POS'].isin(pos_seed)]
            df_check = df[df['POS'].isin(pos_seed)]
            df_test.drop([df_test.columns[9],'match'], axis=1, inplace=True)

            df_test.to_csv(os.path.join(output_dir, f'matched/test/matched_bk{bk}_sc{sc}_chr{chrom}_test.vcf.gz'), sep='\t', index=False)
            df_check.to_csv(os.path.join(output_dir, f'matched/check/matched_bk{bk}_sc{sc}_chr{chrom}_check.vcf.gz'), sep='\t', index=False)

            num_start = df.shape[0]
            num_end = df_test.shape[0]

            dic_table = {'chromosome':chrom, 'bulk':bk, 'sing':sc,
                        'num_start':num_start, 'num_end':num_end, 'removed':(num_start - num_end)}
            table.append(dic_table)

pd.DataFrame(table).to_excel(os.path.join(result_dir, f'matched/general_statistics_matched.xlsx'), index=False)
# dataframe con il numero di snps considerate prima e dopo il mascheramento

In [ ]:
df3 = pd.read_csv(os.path.join(output_dir, f'matched/matched_bk1_sc129_chr1_test.vcf.gz'), header=0, sep='\t', dtype='object')
df4 = pd.read_csv(os.path.join(output_dir, f'matched/matched_bk1_sc129_chr1_check.vcf.gz'), header=0, sep='\t', dtype='object')

In [ ]:
import subprocess

def impute_seed(chrom, bk, sc):
    ref = f'/home/mreverenna/reference/chr{chrom}.1kg.phase3.v5a.vcf.gz '
    map = f'/home/mreverenna/map/plink.chr{chrom}.GRCh37.map '
    gt = f'/home/mreverenna/analysis/vcf_experiments/masking_matching/test/matched_bk{bk}_sc{sc}_chr{chrom}_test.vcf.gz '
    out = f'/home/mreverenna/analysis/vcf_experiments/masking_matching/imputed/imputed_bk{bk}_sc{sc}_chr{chrom}_test'
    positions = f'/home/mreverenna/analysis/vcf_experiments/masking_seeds/positions/file_diff_{chrom}.txt '

    print(f'Imputation of matched_bk{bk}_sc{sc}_chr{chrom}_test is started...')
    command = "java -jar /home/mreverenna/programs/beagle.22Jul22.46e.jar " + f"excludemarkers={positions}" + f"ref={ref}" + f"gt={gt}" + f"map={map}" + f"out={out}"
    result = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE).communicate()

In [ ]:
for bk in bk_list:
    for sc in sc_list:
        for chrom in chromosomes_int:
            impute_seed(chrom, bk, sc)

In [ ]:
df_imputed_1 = pd.read_csv(os.path.join(output_dir, f'imputed_matched_bk1_sc129_chr2_test.vcf.gz'), header=8,sep='\t', dtype='object')
df_imputed_1.iloc[:,9] = df_imputed_1.iloc[:,9].str.split(':').str.get(0)
#sc_merg_chr_imputed = sc_merg_chr_imputed[~sc_merg_chr_imputed.iloc[:, 9:].isin(['3|0', '0|3', '3|3']).any(axis=1)]
df_imputed_1.iloc[:,9].unique()

In [ ]:
wrong_values = ['./.', '.', '1', '0']
valid_values = ['1/1', '0/0', '0/1', '2/2', '1/2']

gen_dosage = {'0/0': 0, '0/1': 1, '1/1': 2, '1/2': 3, '2/2': 4}
imp_dosage = {'0|0': 0, '0|1': 1, '1|0': 1, '1|1': 2, '1|2': 3, '2|1': 3, '2|2': 4}

# values imputed not present in snps array which alterate the jaccard score calculation
not_consider = ['0|2', '2|0','3|0', '0|3', '3|3']


dict_matching = []
for chrom in chromosomes_str:
    dict_post = []
    for bk in bk_list:
        for sc in sc_list:
        
            # load vcfs
            df_check = pd.read_csv(os.path.join(output_dir, f'matched/matched_bk{bk}_sc{sc}_chr{chrom}_check.vcf.gz'), header=0,sep='\t', dtype='object')
            df_imputed = pd.read_csv(os.path.join(output_dir, f'imputed_matched_bk{bk}_sc{sc}_chr{chrom}_test.vcf.gz'), header=8,sep='\t', dtype='object')

            # filter imputed vcf
            df_imputed_check = df_imputed[df_imputed['POS'].isin(df_check['POS'].tolist())]
            df_imputed_check.iloc[:,9] = df_imputed_check.iloc[:,9].str.split(':').str.get(0)
            df_imputed_check = df_imputed_check[~df_imputed_check.iloc[:, 9].isin(not_consider)]
            df_imputed_check.drop_duplicates(subset=['POS'], inplace=True) 

            # filter check vf
            df_check_final = df_check[df_check['POS'].isin(df_imputed_check['POS'].tolist())]

            # check
            print(f'matched_bk{bk}_sc{sc}_chr{chrom}_check.vcf.gz vs imputed_matched_bk{bk}_sc{sc}_chr{chrom}_test')
            print(df_check.shape[0], df_imputed_check.shape[0],'---',df_check_final.shape[0], df_imputed_check.shape[0])

            # calculate jaccard score
            vec_bulk = df_check_final.iloc[:,9].map(gen_dosage).tolist()
            vec_sing = df_imputed_check.iloc[:,9].map(imp_dosage).tolist()

            j_value = jaccard_score(vec_bulk, vec_sing, average='micro')
            dict_matching.append(j_value)

            dict_js = {'sc' : sc, 'bk' : bk, 'chromosome' : chrom, 'j_score': j_value}
            
            dict_post.append(dict_js)

    pd.DataFrame(dict_post).to_excel(os.path.join(result_dir, f'matched/jaccard_scores_chr{chrom}_micro_matched.xlsx'), index=False)
    # append to a new dataframe
            

#### Checking

### Mismatched analysis

In [ ]:
pd.read_csv(os.path.join(output_dir, f'mismatc/GM12878_mismatc_bk1_sc129_chrom1.vcf.gz'), header=0, sep='\t', dtype='object').iloc[:,10].unique()

In [ ]:
perc = 10
table = []

for bk in bk_list:
    for sc in sc_list:
        for chrom in chromosomes_str:

            df = pd.read_csv(os.path.join(output_dir, f'mismatc/GM12878_mismatc_bk{bk}_sc{sc}_chrom{chrom}.vcf.gz'), header=0, sep='\t', dtype='object')
            
            rd.seed(1)
            pos = df['POS'].tolist()
            pos_seed = rd.sample(pos, round((len(pos)*(int(perc)/100))))

            position_masked = os.path.join(pos_dir, f'mismatc/pos_masked_mismatched_{bk}_{sc}_chr{chrom}.txt')
            with open(position_masked,'w') as output:
                output.write(str(pos_seed))

            # creo i dataframe che mi servono da testare e check
            df_test = df[~df['POS'].isin(pos_seed)]
            df_check = df[df['POS'].isin(pos_seed)]
            df_test.drop([df_test.columns[9],'match'], axis=1, inplace=True)

            df_test.to_csv(os.path.join(output_dir, f'mismatc/test/GM12878_mismatc_bk{bk}_sc{sc}_chr{chrom}_test.vcf.gz'), sep='\t', index=False)
            df_check.to_csv(os.path.join(output_dir, f'mismatc/check/GM12878_mismatc_bk{bk}_sc{sc}_chr{chrom}_check.vcf.gz'), sep='\t', index=False)

            num_start = df.shape[0]
            num_end = df_test.shape[0]

            dic_table = {'chromosome':chrom, 'bulk':bk, 'sing':sc, 'num_start':num_start, 'num_end':num_end, 'removed':(num_start - num_end)}
            table.append(dic_table)

pd.DataFrame(table).to_excel(os.path.join(result_dir, f'mismatc/general_statistics_mismatched.xlsx'), index=False)

In [ ]:
df_mismatc = pd.read_csv(os.path.join(output_dir, f'mismatc/GM12878_mismatc_bk1_sc129_chrom1.vcf.gz'), header=0, sep='\t', dtype='object')
df_mismatc.head(3)

In [ ]:
rd.seed(1)
pos = df_mismatc['POS'].tolist()
pos_seed = rd.sample(pos, round((len(pos)*(int(perc)/100))))
len(pos_seed)
#position_masked = os.path.join(pos_dir, f'pos_masked_mismatched_{bk}_{sc}_chr{chrom}.txt')
#with open(position_masked,'w') as output:
#    output.write(str(pos_seed))

In [ ]:
# creo i dataframe che mi servono da testare e check
df_test = df_mismatc[~df_mismatc['POS'].isin(pos_seed)]
df_check = df_mismatc[df_mismatc['POS'].isin(pos_seed)]
df_test.drop([df_test.columns[9],'match'], axis=1, inplace=True)

In [ ]:
df_test.head(3)

In [ ]:
df_check.head(3)

In [ ]:
df_test.to_csv(os.path.join(output_dir, f'mismatc/test/GM12878_mismatc_bk1_sc129_chr1_test.vcf.gz'), sep='\t', index=False)
df_check.to_csv(os.path.join(output_dir, f'mismatc/check/GM12878_mismatc_bk1_sc129_chr1_check.vcf.gz'), sep='\t', index=False)

num_start = df_mismatc.shape[0]
num_end = df_test.shape[0]

In [ ]:
pd.read_csv(os.path.join(output_dir, f'mismatc/imputed/GM12878_mismatc_imputed_bk1_sc129_chr1_test.vcf.gz'), header=8,sep='\t', dtype='object').head(3)

In [ ]:
wrong_values = ['./.', '.', '1', '0']
valid_values = ['1/1', '0/0', '0/1', '2/2', '1/2']
bk_list = ['1']
gen_dosage = {'0/0': 0, '0/1': 1, '1/1': 2, '1/2': 3, '2/2': 4}
imp_dosage = {'0|0': 0, '0|1': 1, '1|0': 1, '1|1': 2, '1|2': 3, '2|1': 3, '2|2': 4}

# values imputed not present in snps array which alterate the jaccard score calculation
not_consider = ['0|2', '2|0','3|0', '0|3', '3|3']

dict_matching = []
for chrom in chromosomes_str:
    dict_post = []
    for bk in bk_list:
        for sc in sc_list:
        
            # load vcfs
            df_check = pd.read_csv(os.path.join(output_dir, f'mismatc/check/GM12878_mismatc_bk{bk}_sc{sc}_chr{chrom}_check.vcf.gz'), header=0,sep='\t', dtype='object')
            df_imputed = pd.read_csv(os.path.join(output_dir, f'mismatc/imputed/GM12878_mismatc_imputed_bk{bk}_sc{sc}_chr{chrom}_test.vcf.gz'), header=8,sep='\t', dtype='object')

            # filter imputed vcf
            df_imputed_check = df_imputed[df_imputed['POS'].isin(df_check['POS'].tolist())]
            df_imputed_check.iloc[:,9] = df_imputed_check.iloc[:,9].str.split(':').str.get(0)
            df_imputed_check = df_imputed_check[~df_imputed_check.iloc[:, 9].isin(not_consider)]
            df_imputed_check.drop_duplicates(subset=['POS'], inplace=True) 

            # filter check vf
            df_check_final = df_check[df_check['POS'].isin(df_imputed_check['POS'].tolist())]

            # check
            print(f'matched_bk{bk}_sc{sc}_chr{chrom}_check.vcf.gz vs imputed_matched_bk{bk}_sc{sc}_chr{chrom}_test')
            print(df_check.shape[0], df_imputed_check.shape[0])
            print(df_check_final.shape[0], df_imputed_check.shape[0])

            # calculate jaccard score
            vec_bulk = df_check_final.iloc[:,9].map(gen_dosage).tolist()
            vec_sing = df_imputed_check.iloc[:,9].map(imp_dosage).tolist()

            j_value = jaccard_score(vec_bulk, vec_sing, average='micro')
            dict_matching.append(j_value)

            dict_js = {'sc' : sc, 'bk' : bk, 'chromosome' : chrom, 'j_score': j_value}
            
            dict_post.append(dict_js)

    pd.DataFrame(dict_post).to_excel(os.path.join(result_dir, f'mismatc/jaccard_scores_chr{chrom}_micro_mismatched.xlsx'), index=False)
    # append to a new dataframe
            

### Second matching

In [ ]:
sc_list

In [ ]:
for bk in bk_list:
    for sc in sc_list:
        for chrom in chromosomes_str:
            df_bk = pd.read_csv(f'data/GM12878_gDNA_{bk}.vcf', header=30,sep='\t', dtype='object')
            df_sc = pd.read_csv(f'data/GM12878_SC_{sc}.vcf.gz', header=30,sep='\t', dtype='object')

            merged_df = pd.concat([df_bk, df_sc.iloc[:,9]], axis=1)
            merged_df_auto = merged_df[merged_df.iloc[:,0].isin(chromosomes_str)]
            
            merged_df_auto.iloc[:, 9] = merged_df_auto.iloc[:, 9].str.split(':').str[0]
            merged_df_auto.iloc[:, 10] = merged_df_auto.iloc[:, 10].str.split(':').str[0]
            
            merged_df_auto['FORMAT'] = 'GT'

            merged_df_auto.drop_duplicates(subset = ['POS'], inplace =True)

            merged_df_auto = merged_df_auto[~merged_df_auto.iloc[:,9].isin(wrong_values)]

            merged_df_auto['second_match'] = (merged_df_auto.iloc[:, 9] == merged_df_auto.iloc[:, 10]).astype(int)

            sc_test = merged_df_auto[merged_df_auto['second_match']== 1].drop(columns=merged_df_auto.columns[[9,11]])
            bk_check = merged_df_auto[merged_df_auto['second_match']== 0].drop(columns=merged_df_auto.columns[[10,11]])

            #if './.' in bk_check.iloc[:, 9].unique():
            #    raise ValueError("Error: Column 9 of gDNA contains './.' among the unique positions and cannot contain missing values")
            #elif './.' not in sc_test.iloc[:, 9].unique():
            #    raise ValueError("Error: Column 9 of scDNA does not contain './.' among the unique positions and should contain missing values")
            
            sc_test_chr = sc_test[sc_test['#CHROM'] == chrom]
            bk_check_chr = bk_check[bk_check['#CHROM'] == chrom]

            sc_test_chr.to_csv(os.path.join(output_dir, f'second/test/GM12878_bk{bk}_sc{sc}_chr{chrom}_test.vcf'), sep='\t', index=False)
            bk_check_chr.to_csv(os.path.join(output_dir, f'second/check/GM12878_bk{bk}_sc{sc}_chr{chrom}_check.vcf'), sep='\t', index=False)    

In [ ]:
df_bk = pd.read_csv(os.path.join(output_dir, f'second/check/GM12878_bk1_sc129_chr1_check.vcf'), header=0,sep='\t', dtype='object')

In [ ]:
df_bk

In [ ]:
wrong_values = ['./.', '.', '1', '0']
valid_values = ['1/1', '0/0', '0/1', '2/2', '1/2']
bk_list = ['1']
gen_dosage = {'0/0': 0, '0/1': 1, '1/1': 2, '1/2': 3, '2/2': 4}
imp_dosage = {'0|0': 0, '0|1': 1, '1|0': 1, '1|1': 2, '1|2': 3, '2|1': 3, '2|2': 4}

# values imputed not present in snps array which alterate the jaccard score calculation
not_consider = ['0|2', '2|0','3|0', '0|3', '3|3']

dict_second = []
for chrom in chromosomes_str:
    dict_post = []
    for bk in bk_list:
        for sc in sc_list:
        
            # load vcfs
            df_check = pd.read_csv(os.path.join(output_dir, f'second/check/GM12878_bk{bk}_sc{sc}_chr{chrom}_check.vcf'), header=0,sep='\t', dtype='object')
            df_imputed = pd.read_csv(os.path.join(output_dir, f'second/imputed/GM12878_second_imputed_bk{bk}_sc{sc}_chr{chrom}_test.vcf.gz'), header=8,sep='\t', dtype='object')

            # filter imputed vcf
            df_imputed_check = df_imputed[df_imputed['POS'].isin(df_check['POS'].tolist())]
            df_imputed_check.iloc[:,9] = df_imputed_check.iloc[:,9].str.split(':').str.get(0)
            df_imputed_check = df_imputed_check[~df_imputed_check.iloc[:, 9].isin(not_consider)]
            df_imputed_check.drop_duplicates(subset=['POS'], inplace=True) 

            # filter check vf
            df_check_final = df_check[df_check['POS'].isin(df_imputed_check['POS'].tolist())]

            # check
            #print(f'matched_bk{bk}_sc{sc}_chr{chrom}_check.vcf.gz vs imputed_matched_bk{bk}_sc{sc}_chr{chrom}_test')
            #print(df_check.shape[0], df_imputed_check.shape[0])
            #print(df_check_final.shape[0], df_imputed_check.shape[0])

            # calculate jaccard score
            vec_bulk = df_check_final.iloc[:,9].map(gen_dosage).tolist()
            vec_sing = df_imputed_check.iloc[:,9].map(imp_dosage).tolist()

            j_value = jaccard_score(vec_bulk, vec_sing, average='micro')
            dict_second.append(j_value)

            dict_js = {'sc' : sc, 'bk' : bk, 'chromosome' : chrom, 'j_score': j_value}
            
            dict_post.append(dict_js)

    pd.DataFrame(dict_post).to_excel(os.path.join(result_dir, f'second/jaccard_scores_chr{chrom}_micro_second.xlsx'), index=False)
    # append to a new dataframe
            

In [ ]:
# load excels jaccard score
file_names = [os.path.join(result_dir,f'second/jaccard_scores_chr{chrom}_micro_second.xlsx') for chrom in chromosomes_str]
dfs = [pd.read_excel(file) for file in file_names] # read each file into a list of DataFrames
result = pd.concat(dfs)

grouped_data = result.groupby('chromosome').mean().reset_index()

In [ ]:
fig = px.scatter(grouped_data, x="chromosome", y="j_score",
                 title="Jaccard scores by chromosome - second genotypes - grouped by chromosome (1bk 5sc, mean)",
                 labels={"salary":"Annual Salary (in thousands)"} # customize axis label
                )
fig.update_traces(marker_size=10)
fig.update_layout(yaxis=dict(range=[0.992, 1]),
                             xaxis=dict(tickmode='array', tickvals=np.arange(1, 23)),
                             yaxis_title='Jaccard scores',
                             xaxis_title='Chromosomes')
fig.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6)) # set the figure size
ax = sns.scatterplot(data=grouped_data, x='chromosome', y='j_score')
ax.set_title('Jaccard scores by chromosome considering matched genotypes')
# took same SNPs, masking 10% and impute back
ax.set_xlabel('Chromosomes')
ax.set_ylabel('Jaccard scores')
ax.set_xticks(range(1, len(grouped_data) + 1))
#plt.show()
plt.savefig(os.path.join(plot_dir, 'matched_scatterplot_all_chromosomes.pdf'))

In [ ]:
df_bk.shape

In [ ]:
df_sc = pd.read_csv(os.path.join(output_dir, f'second/test/GM12878_bk1_sc129_chr1_test.vcf'), header=0,sep='\t', dtype='object')

In [ ]:
df_sc.iloc[:,9].unique()

In [ ]:
df_sc.shape

In [ ]:
merged_df = pd.concat([bk1, sc129.iloc[:,9]], axis=1)

merged_df_auto = merged_df[merged_df.iloc[:,0].isin(chromosomes_str)]
merged_df_auto.iloc[:, 9] = merged_df_auto.iloc[:, 9].str.split(':').str[0]
merged_df_auto.iloc[:, 10] = merged_df_auto.iloc[:, 10].str.split(':').str[0]
merged_df_auto['FORMAT'] = 'GT'

merged_df_auto.drop_duplicates(subset = ['POS'], inplace =True)

merged_df_auto = merged_df_auto[~merged_df_auto.iloc[:,9].isin(wrong_values)]


merged_df_auto['second_match'] = (merged_df_auto.iloc[:, 9] == merged_df_auto.iloc[:, 10]).astype(int)


sc_test = merged_df_auto[merged_df_auto['second_match']== 1].drop(columns=merged_df_auto.columns[[9,11]])
bk_check = merged_df_auto[merged_df_auto['second_match']== 0].drop(columns=merged_df_auto.columns[[10,11]])

pos_check = bk_check.POS.tolist()
#merged_df_auto.POS.value_counts()

In [ ]:
# Verifica se la colonna 9 del DataFrame bk_check contiene './.' tra le posizioni uniche
if './.' in bk_check.iloc[:, 9].unique():
    raise ValueError("Errore: la colonna 9 del DataFrame bk_check contiene './.' tra le posizioni uniche e non può contenere missing values")

# Verifica se la colonna 9 del DataFrame sc_test non contiene './.' tra le posizioni uniche
if './.' in sc_test.iloc[:, 9].unique():
    raise ValueError("Errore: la colonna 9 del DataFrame sc_test non contiene './.' tra le posizioni uniche e deve contenere missing values")


In [ ]:
bk_check.iloc[:,9].unique()

In [ ]:
sc_test.iloc[:,9].unique()

In [ ]:
df_bk = bk1[bk1.iloc[:,0].isin(chromosomes_str)]
df_sing = sc129[sc129.iloc[:,0].isin(chromosomes_str)]

In [ ]:
df_bk_filt = df_bk[~df_bk.iloc[:,9].isin(wrong_values)]

In [ ]:
df_bk_filt

In [ ]:
df_bk_filt.iloc[:,9].unique()

In [ ]:
df_bk_filt.shape

In [ ]:
df_bk_filt.drop_duplicates(subset=['POS'], inplace=True) 
df_bk_filt.shape

In [ ]:
pos_bk_filt = df_bk_filt.POS.tolist()

In [ ]:
pos_bk_filt

In [ ]:
df_sc_filt = df_sing[df_sing['POS'].isin(pos_bk_filt)]

In [ ]:
df_sc_filt.drop_duplicates(subset=['POS'], inplace=True) 

In [ ]:
df_sc_filt

In [ ]:
df_sc_filt.iloc[:, 9] = df_sc_filt.iloc[:, 9].str.split(':').str[0]

In [ ]:
print(df_sc_filt.shape)
print(df_bk_filt.shape)

In [ ]:
df_sc_filt

In [ ]:
merged_df = pd.concat([df_bk_filt, df_sc_filt.iloc[:,9]], axis=1)

In [ ]:
df_sc_filt[df_sc_filt.index == 28588]

In [ ]:
merged_df

In [ ]:
wrong_values = ['./.','.', '1', '0']

for bk in bk_list:
    for sc in sc_list:
        
        print(f'starting merging datasets bulk {bk} and single cell {sc}')
        df_bk = pd.read_csv(f'data/GM12878_gDNA_{bk}.vcf', header=30,sep='\t', dtype='object')
        df_sing = pd.read_csv(f'data/GM12878_SC_{sc}.vcf.gz', header=30,sep='\t', dtype='object')

        column_names = df_bk.columns.to_list()
        
        df_bk = df_bk[df_bk.iloc[:,0].isin(chromosomes_str)]
        df_sing = df_sing[df_sing.iloc[:,0].isin(chromosomes_str)]

        df_bk.iloc[:, 9] = df_bk.iloc[:, 9].str.split(':').str[0]

        df_bk_filt = df_bk[~df_bk.iloc[:,9].isin(wrong_values)]
        
        pos_bk_filt = df_bk_filt.POS.tolist()

        df_sc_filt = df_sing[df_sing['POS'].isin(pos_bk_filt)]

        merged_df = pd.concat([df_bk_filt, df_sc_filt.iloc[:,9]], axis=1)


        #merged_df.iloc[:,9] = merged_df.iloc[:,9].apply(lambda x: x.split(':')[0])
        #merged_df.iloc[:,10] = merged_df.iloc[:,10].apply(lambda x: x.split(':')[0])

        merged_df['FORMAT'] = 'GT'

        merged_df['match'] = (merged_df.iloc[:,9] == merged_df.iloc[:,10]).astype(int)

        valid_values = ['1/1', '0/0', '0/1', '2/2', '1/2']
        merged_df_filt = merged_df[merged_df.iloc[:,9].isin(valid_values)]
        
        print(merged_df_filt.shape[0])

        merged_df_filt_test = merged_df_filt[merged_df_filt['match']==1] # tenere bulk 9
        merged_df_filt_check = merged_df_filt[merged_df_filt['match']==0] # tenere sc 10

        
#####
        merged_df_filt_test.to_csv(os.path.join(output_dir, f'second/GM12878_matched_bk{bk}_sc{sc}_chrs_test.vcf'), sep='\t', index=False)

        #print(f'dimension before filtering: {merged_df.shape[0]}')

        #merged_df['match'] = (merged_df.iloc[:,9] == merged_df.iloc[:,10]).astype(int)

        #valid_values = ['1/1', '0/0', '0/1', '2/2', '1/2']
        #merged_df_filt = merged_df[merged_df.iloc[:,9].isin(valid_values)]
        

        #print(merged_df_filt.shape[0])

        #merged_df_filt_matched = merged_df_filt[merged_df_filt['match']==1]
        #merged_df_filt_mismatc = merged_df_filt[merged_df_filt['match']==0]

        #merged_df_filt_matched.to_csv(os.path.join(output_dir, f'matched/GM12878_matched_bk{bk}_sc{sc}_chrs.vcf'), sep='\t', index=False)
        #merged_df_filt_mismatc.to_csv(os.path.join(output_dir, f'mismatc/GM12878_mismatc_bk{bk}_sc{sc}_chrs.vcf.gz'), sep='\t', index=False)

        #print('merging datasets finished')

        #for chrom in chromosomes_str:
            # splitting chromosomes
            
        #    merged_df_filt_matched_chr = merged_df_filt_matched[merged_df_filt_matched['#CHROM'] == chrom]
        #    merged_df_filt_mismatc_chr = merged_df_filt_mismatc[merged_df_filt_mismatc['#CHROM'] == chrom]

        #    merged_df_filt_matched_chr.to_csv(os.path.join(output_dir, f'matched/GM12878_matched_bk{bk}_sc{sc}_chrom{chrom}.vcf'), sep='\t', index=False)
        #    merged_df_filt_mismatc_chr.to_csv(os.path.join(output_dir, f'mismatc/GM12878_mismatc_bk{bk}_sc{sc}_chrom{chrom}.vcf.gz'), sep='\t', index=False)

In [ ]:
pd.read_csv(os.path.join(output_dir, f'second/GM12878_matched_bk1_sc129_chrs_test.vcf'), header=0,sep='\t', dtype='object')

In [ ]:
sc_merg_chr_imputed = sc_merg_chr_imputed[~sc_merg_chr_imputed.iloc[:, 9:].isin(['./.']).any(axis=1)]

In [ ]:
perc = 10
table = []

for bk in bk_list:
    for sc in sc_list:
        for chrom in chromosomes_str:

            df = pd.read_csv(os.path.join(output_dir, f'mismatc/GM12878_mismatc_bk{bk}_sc{sc}_chrom{chrom}.vcf.gz'), header=0, sep='\t', dtype='object')
            
            rd.seed(1)
            pos = df['POS'].tolist()
            pos_seed = rd.sample(pos, round((len(pos)*(int(perc)/100))))

            position_masked = os.path.join(pos_dir, f'mismatc/pos_masked_mismatched_{bk}_{sc}_chr{chrom}.txt')
            with open(position_masked,'w') as output:
                output.write(str(pos_seed))

            # creo i dataframe che mi servono da testare e check
            df_test = df[~df['POS'].isin(pos_seed)]
            df_check = df[df['POS'].isin(pos_seed)]
            df_test.drop([df_test.columns[9],'match'], axis=1, inplace=True)

            df_test.to_csv(os.path.join(output_dir, f'mismatc/test/GM12878_mismatc_bk{bk}_sc{sc}_chr{chrom}_test.vcf.gz'), sep='\t', index=False)
            df_check.to_csv(os.path.join(output_dir, f'mismatc/check/GM12878_mismatc_bk{bk}_sc{sc}_chr{chrom}_check.vcf.gz'), sep='\t', index=False)

            num_start = df.shape[0]
            num_end = df_test.shape[0]

            dic_table = {'chromosome':chrom, 'bulk':bk, 'sing':sc, 'num_start':num_start, 'num_end':num_end, 'removed':(num_start - num_end)}
            table.append(dic_table)

pd.DataFrame(table).to_excel(os.path.join(result_dir, f'mismatc/general_statistics_mismatched.xlsx'), index=False)

### Plots

In [ ]:
# load excels jaccard score
file_names = [os.path.join(result_dir,f'matched/jaccard_scores_chr{chrom}_micro_matched.xlsx') for chrom in chromosomes_str]
dfs = [pd.read_excel(file) for file in file_names] # read each file into a list of DataFrames
result = pd.concat(dfs)

In [ ]:
result.head(20)

In [ ]:
grouped_data = result.groupby('chromosome').mean().reset_index()

In [ ]:
grouped_data

In [ ]:
fig = px.scatter(grouped_data, x="chromosome", y="j_score",
                 title="Jaccard scores by chromosome - matched genotypes - grouped by chromosome (1bk 5sc, mean)",
                 labels={"salary":"Annual Salary (in thousands)"} # customize axis label
                )
fig.update_traces(marker_size=10)
fig.update_layout(yaxis=dict(range=[0.992, 1]),
                             xaxis=dict(tickmode='array', tickvals=np.arange(1, 23)),
                             yaxis_title='Jaccard scores',
                             xaxis_title='Chromosomes')
fig.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6)) # set the figure size
ax = sns.scatterplot(data=grouped_data, x='chromosome', y='j_score')
ax.set_title('Jaccard scores by chromosome considering matched genotypes')
# took same SNPs, masking 10% and impute back
ax.set_xlabel('Chromosomes')
ax.set_ylabel('Jaccard scores')
ax.set_xticks(range(1, len(grouped_data) + 1))
#plt.show()
plt.savefig(os.path.join(plot_dir, 'matched_scatterplot_all_chromosomes.pdf'))

In [ ]:
# load excels jaccard score
file_names = [os.path.join(result_dir,f'mismatc/jaccard_scores_chr{chrom}_micro_mismatched.xlsx') for chrom in chromosomes_str]
dfs = [pd.read_excel(file) for file in file_names] # read each file into a list of DataFrames
result = pd.concat(dfs)

grouped_data = result.groupby('chromosome').mean().reset_index()

fig, ax = plt.subplots(figsize=(8, 6)) # set the figure size
ax = sns.scatterplot(data=grouped_data, x='chromosome', y='j_score')
ax.set_title('Jaccard scores by chromosome considering mismatched genotypes')
# took same SNPs, masking 10% and impute back
ax.set_xlabel('Chromosomes')
ax.set_ylabel('Jaccard scores')
ax.set_xticks(range(1, len(grouped_data) + 1))
#plt.show()
plt.savefig(os.path.join(plot_dir, 'mismatched_scatterplot_all_chromosomes.pdf'))
#plt.savefig()